In [7]:
import sys
sys.path.append('/Users/yuanqingwang/Documents/GitHub/mudslide/')

In [8]:
import mudslide

In [9]:
import openmm as mm
import numpy as onp
import jax
import jax.numpy as jnp

In [10]:

openmm_system = mm.System()
openmm_system.addParticle(1.0)
openmm_system.addParticle(1.0)
force = mm.HarmonicBondForce()
force.addBond(0, 1.0, 0.0, 1.0)
openmm_system.addForce(force)

0

In [11]:
X = onp.random.randn(2, 3)
V = onp.random.randn(2, 3)

In [35]:
from openmmtools.integrators import VelocityVerletIntegrator
integrator = VelocityVerletIntegrator(0.1 * mm.unit.picosecond)
context = mm.Context(openmm_system, integrator)
context.setPositions(X * mm.unit.nanometer)
context.setVelocities(V * mm.unit.nanometer/mm.unit.picosecond)

In [36]:
%%timeit
integrator.step(100)

2.15 ms ± 76.3 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [32]:
from mudslide.integrators.velocity_verlet import VelocityVerletIntegrator
from mudslide.system import System
from mudslide.state import State
from mudslide.forces.harmonic_bond_force import HarmonicBondForce
force = HarmonicBondForce(
    particle1=jnp.array([0]), 
    particle2=jnp.array([1]), 
    length=jnp.array([0.0]), 
    k=jnp.array([1.0]),
)
system = System(masses=jnp.array([1.0, 1.0]), forces=[force])
integrator = VelocityVerletIntegrator(timestep=0.1)
integrate = jax.jit(lambda state: integrator(100, state, system, key=jax.random.PRNGKey(0)))

In [33]:
%%timeit
state = State(jnp.array(X), jnp.array(V))
state = integrate(state)
jax.block_until_ready(state)

61.1 μs ± 6.04 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
